<a href="https://colab.research.google.com/github/shuwang127/636Project/blob/main/Classifers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np

_COLAB_ = 0 if (os.getenv('COLAB_GPU', 'NONE') == 'NONE') else 1
rootPath = './drive/My Drive/Colab Notebooks/' if (_COLAB_) else './'

In [2]:
# read data.
data = pd.read_csv(rootPath + 'training_data.csv')
# data.head(3)

In [3]:
# fill missing features.
data = data.replace(' ?', np.nan)
for index, row in data.iteritems():
    data[index].fillna(data[index].mode()[0], inplace=True)
# data.head(3)

In [4]:
# get labels.
Y = data['salary'].replace([' <=50K' , ' >50K'] , [0, 1])
# Y.head(3)

In [5]:
# get features.
X = data.drop(['ID (this is not a feature)', 'salary'], axis=1)
# normalize numeric features.
from sklearn.preprocessing import StandardScaler
numcol = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
scaler = StandardScaler()
X[numcol] = scaler.fit_transform(X[numcol])
# get one-hot vectors.
X = pd.get_dummies(X)
# X.head(3)

In [6]:
# separate training data and validation data.
from sklearn.model_selection import train_test_split
XTrain, XTest, YTrain, YTest = train_test_split(X, Y, test_size=0.2)
print("size of training data is", XTrain.shape)
print("size of testing data is", XTest.shape)

size of training data is (31073, 105)
size of testing data is (7769, 105)


In [7]:
# evaluation.
from sklearn.metrics import accuracy_score, confusion_matrix
def Evaluation(YTest, YPred, method=''):
    print('====================== ' + method + ' ======================')
    acc = accuracy_score(YTest, YPred) * 100
    print('Accuracy is %.3f%%.' % acc)
    conf = confusion_matrix(YTest, YPred)
    print(pd.DataFrame(conf, columns=['Pred-Neg','Pred-Pos'], index=['Actl-Neg', 'Actl-Pos']))
    precision = conf[1][1] / (conf[0][1] + conf[1][1]) if (conf[0][1] + conf[1][1]) else 0
    recall = conf[1][1] / (conf[1][0] + conf[1][1]) if (conf[1][0] + conf[1][1]) else 0
    F1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0
    print('Precision is %.3f.' % precision)
    print('Recall is %.3f.' % recall)
    print('F1 score is %.3f.' % F1)

In [8]:
# NaiveBayes.
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
Evaluation(YTest, YPred, 'Naive Bayes')

====================== Naive Bayes ======================
Accuracy is 57.665%.
          Pred-Neg  Pred-Pos
Actl-Neg      2746      3177
Actl-Pos       112      1734
Precision is 0.353.
Recall is 0.939.
F1 score is 0.513.


In [9]:
# Least Square Regression.
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
YPred = (YPred > 0.5)
Evaluation(YTest, YPred, 'Least Square Regression')

====================== Least Square Regression ======================
Accuracy is 83.898%.
          Pred-Neg  Pred-Pos
Actl-Neg      5604       319
Actl-Pos       932       914
Precision is 0.741.
Recall is 0.495.
F1 score is 0.594.


In [10]:
# Ridge Regression.
from sklearn.linear_model import Ridge
model = Ridge(alpha=1e-2, normalize=True)
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
YPred = (YPred > 0.5)
Evaluation(YTest, YPred, 'Ridge Regression')

====================== Ridge Regression ======================
Accuracy is 83.923%.
          Pred-Neg  Pred-Pos
Actl-Neg      5607       316
Actl-Pos       933       913
Precision is 0.743.
Recall is 0.495.
F1 score is 0.594.


In [11]:
# Lasso Regression.
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
model = Lasso(alpha=1e-6)
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
YPred = (YPred > 0.5)
Evaluation(YTest, YPred, 'Lasso Regression')

====================== Lasso Regression ======================
Accuracy is 83.910%.
          Pred-Neg  Pred-Pos
Actl-Neg      5605       318
Actl-Pos       932       914
Precision is 0.742.
Recall is 0.495.
F1 score is 0.594.


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1533.8947350060726, tolerance: 0.567938853667163
  positive)


In [12]:
# Logistic Regression.
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
Evaluation(YTest, YPred, 'Logistic Regression')

====================== Logistic Regression ======================
Accuracy is 85.223%.
          Pred-Neg  Pred-Pos
Actl-Neg      5529       394
Actl-Pos       754      1092
Precision is 0.735.
Recall is 0.592.
F1 score is 0.655.


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [13]:
# K-means.
from sklearn.cluster import KMeans
model = KMeans(n_clusters=2, random_state=0)
model.fit(X)
YPred = model.labels_
import scipy.stats as stats
if (stats.pearsonr(Y, YPred)[0] < 0):
    YPred = (YPred == 0)
Evaluation(Y, YPred, 'K-means')

====================== K-means ======================
Accuracy is 66.951%.
          Pred-Neg  Pred-Pos
Actl-Neg     17521     11995
Actl-Pos       842      8484
Precision is 0.414.
Recall is 0.910.
F1 score is 0.569.


In [14]:
# Decision Tree.
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(criterion="gini", splitter="best", max_features=None, min_samples_split=300, min_samples_leaf=1, random_state=0)
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
Evaluation(YTest, YPred, 'Decision Tree')

====================== Decision Tree ======================
Accuracy is 85.622%.
          Pred-Neg  Pred-Pos
Actl-Neg      5536       387
Actl-Pos       730      1116
Precision is 0.743.
Recall is 0.605.
F1 score is 0.666.


In [15]:
# Random Forest.
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, max_depth=100, criterion='gini', max_features = 'sqrt', n_jobs=-1, verbose=1, random_state=0)
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
Evaluation(YTest, YPred, 'Random Forest')

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s


====================== Random Forest ======================
Accuracy is 85.236%.
          Pred-Neg  Pred-Pos
Actl-Neg      5481       442
Actl-Pos       705      1141
Precision is 0.721.
Recall is 0.618.
F1 score is 0.666.


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.2s finished


In [16]:
# Support Vector Machine.
from sklearn.svm import SVC
model = SVC(kernel='linear', C=1)
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
Evaluation(YTest, YPred, 'Support Vector Machine')

====================== Support Vector Machine ======================
Accuracy is 84.979%.
          Pred-Neg  Pred-Pos
Actl-Neg      5576       347
Actl-Pos       820      1026
Precision is 0.747.
Recall is 0.556.
F1 score is 0.637.


In [17]:
# Neural Network.
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(32,8), activation='relu', solver='lbfgs', max_iter=100)
model.fit(XTrain, YTrain)
YPred = model.predict(XTest)
Evaluation(YTest, YPred, 'Neural Network')

====================== Neural Network ======================
Accuracy is 85.378%.
          Pred-Neg  Pred-Pos
Actl-Neg      5459       464
Actl-Pos       672      1174
Precision is 0.717.
Recall is 0.636.
F1 score is 0.674.


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
